<a href="https://colab.research.google.com/github/TFremon/CHE-2064/blob/main/Proj4_AnalysisCode_Team135.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [139]:
#@title
#Imports
import math
import numpy as np
from plotly.subplots import make_subplots
from scipy.optimize import minimize
import plotly.io as pio
pio.templates.default='plotly_dark'

In [140]:
#@title
# Data get
!wget https://raw.githubusercontent.com/TFremon/ENED-1120/main/Proj4_Data_Team135.csv


--2022-03-10 20:45:53--  https://raw.githubusercontent.com/TFremon/ENED-1120/main/Proj4_Data_Team135.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 472 [text/plain]
Saving to: ‘Proj4_Data_Team135.csv.6’

Proj4_Data_Team135. 100%[===================>]     472  --.-KB/s    in 0s      

2022-03-10 20:45:53 (24.2 MB/s) - ‘Proj4_Data_Team135.csv.6’ saved [472/472]



In [141]:
#@title
# Get data, create array
data = np.loadtxt('Proj4_Data_Team135.csv', delimiter=',')


In [142]:
#@title
# convert test # to corresponding x/y coords
# create x, y value lists
x = []
y = []
for i in range (len(data)):
  #append y or x for tests 1-4, 5-8
  if data[i][0] > 0 and data[i][0] < 5:
    x.append(0)
    if data[i][0] == 1:
      y.append(12)
    elif data[i][0] == 2:
      y.append(36)
    elif data[i][0] == 3:
      y.append(60)
    elif data[i][0] == 4:
      y.append(84)
  else:
    y.append(12)
    if data[i][0] == 5:
      x.append(12)
    elif data[i][0] == 6:
      x.append(24)
    elif data[i][0] == 7:
      x.append(48)
    elif data[i][0] == 8:
      x.append(96)

#error columns
x_error = data[:,1]
y_error = data[:,2]




In [143]:
#@title
#REMOVE###########################################
#Test data generation
#x error is polynomial
#y error is linear
#x = np.linspace(0, 12, 100)
#y = np.linspace(12, 96, 100)
#x_error = []
#y_error = []
#for i in range(len(x)):
# x_error.append(-np.random.ranf()*x[i]**(np.random.ranf()*3))
#for e in range(len(y)):
#  y_error.append(0.1*-np.random.ranf()*y[e])
#x_error = np.array(x_error)
#y_error = np.array(y_error)

#t_data = [x, y, x_error, y_error]


In [144]:
#@title
#store displacement list
displacement = []
for i in range(len(x)):
  displacement.append((np.sqrt(x[i]**2 + y[i]**2)))


In [160]:
#@title
# create sse functions
# assumes cubic change for both x, y errors
# returns assumes errors are result of total displacement
# first fun is x sse, second return is y sse
def sse_x(v):
  ax, bx, cx, dx = v
  sum = 0
  for i in range(len(displacement)):
    sum = sum + ((ax*displacement[i]**3 + bx*displacement[i]**2 + 
                   cx*displacement[i] + dx - x_error[i])**2)

  return sum

def sse_y_exclude(v):
  ayx, byx, cyx, dyx = v
  sum = 0
  for i in range(len(displacement)):
    if data[i][0] != 4:
      sum = sum + ((ayx*displacement[i]**3 + byx*displacement[i]**2 + 
                   cyx*displacement[i] + dyx - y_error[i])**2)

  return sum

def sse_y(v):
    ay, by, cy, dy = v
    sum = 0
    for i in range(len(displacement)):
      sum = sum + ((ay*displacement[i]**3 + by*displacement[i]**2 + 
                   cy*displacement[i] + dy - y_error[i])**2)

    return sum

In [161]:
#@title
#determine mx, bx, my, b parameters through minimization of sse funs
ax, bx, cx, dx = minimize(sse_x,[0.1, 0.1, 0.1, 0.1]).x
ay, by, cy, dy = minimize(sse_y,[0.1, 0.1, 0.1, 0.1]).x
ayx, byx, cyx, dyx = minimize(sse_y_exclude,[0.1, 0.1, 0.1, 0.1]).x
ay, ayx



(0.00012106377444776113, 9.454659661446474e-05)

In [162]:
#@title
#error prediction using parameters
def error_predict(loc):
  x, y = loc

  return((ax*x**3 + bx*x**2 +cx*x + dy), (ay*y**3 + by*y**2 +cy*y + dy))


In [163]:
#@title
#error models
#linearly spaced plot ofr displacement to better show model
displacement_plot = np.linspace(12, max(displacement), 500)
x_model = (ax*displacement_plot**3 + bx*displacement_plot**2 + 
           cx*displacement_plot + dx)
y_model = (ay*displacement_plot**3 + by*displacement_plot**2 + 
           cy*displacement_plot + dy)
y_model_exclude = (ayx*displacement_plot**3 + byx*displacement_plot**2 + 
           cyx*displacement_plot + dyx)
ayx


  


9.454659661446474e-05

In [164]:
#@title
#calculate average errors, standard deviations overall
#and for each test

average_x_error = np.average(x_error)
average_y_error = np.average(y_error)
std_x_error = np.std(x_error)
std_y_error = np.std(y_error)
averagex_tests = []
averagey_tests = []
stdx_tests = []
stdy_tests = []
#averages of each group of 5
for e in range(0, 40, 5):
  averagex_tests.append(np.average(x_error[e:e+4]))
  averagey_tests.append(np.average(y_error[e:e+4]))
  stdx_tests.append(np.std(x_error[e:e+4]))
  stdy_tests.append(np.std(y_error[e:e+4]))

In [166]:
#@title
#x error model 
#y error model 
figure = make_subplots()
figure.update_layout(title = 'Error Models', xaxis_title = 'Displacement (in)', yaxis_title = 'Error (in)')
figure.add_scatter(x = displacement, y = x_error, mode = 'markers', name = 'x error')
figure.add_scatter(x = displacement_plot, y = x_model, name = 'x error model')
figure.add_scatter(x = displacement, y = y_error, mode = 'markers', name = 'y error')
figure.add_scatter(x = displacement_plot, y = y_model, name = 'y error model', line_color = 'White')
figure.add_scatter(x = displacement_plot, y = y_model_exclude, name = 'y error model- Test 4 excluded', line_color = 'Pink')

#Tests 1-8 average plotting
#Create test plot
test_plot_avg = np.linspace(1, 8, 8)
figure1 = make_subplots()
figure1.update_layout(title = 'Tests 1-8 Average Errors', xaxis_title = 'Test Number', yaxis_title = 'Error Average (in)')
figure1.add_scatter(x = test_plot_avg, y = averagex_tests, mode = 'markers', name = 'x error average')
figure1.add_scatter(x = test_plot_avg, y = averagey_tests, mode = 'markers', name = 'y error averages')
figure1.add_scatter(x = test_plot_avg, y = [average_x_error]*8, mode = 'lines', name = 'x error average all tests')
figure1.add_scatter(x = test_plot_avg, y = [average_y_error]*8, mode = 'lines', name = 'y error average all tests')

#Tests 1-8 Standard Deviation plotting
figure2 = make_subplots()
figure2.update_layout(title = 'Tests 1-8 Standard Deviations', xaxis_title = 'Test Number', yaxis_title = 'Standard Deviation (in)')
figure2.add_scatter(x = test_plot_avg, y = stdx_tests, mode = 'markers', name = 'x error average')
figure2.add_scatter(x = test_plot_avg, y = stdy_tests, mode = 'markers', name = 'y error averages')
figure2.add_scatter(x = test_plot_avg, y = [std_x_error]*8, mode = 'lines', name = 'x error deviation all tests')
figure2.add_scatter(x = test_plot_avg, y = [std_y_error]*8, mode = 'lines', name = 'y error deviation all tests')
display(figure, figure1, figure2)



In [151]:
#@title
x_inp = float(input('Enter x coordinate of box: \n'))
y_inp = float(input('Enter y coordinate of box: \n'))
x_pred, y_pred = error_predict([x_inp, y_inp])
print('The predicted displacement is ({0:.2f}, {1:.2f}) inches \n' .format(x_pred,y_pred))
print('The average error and standard deviation in the x-direction is {0:.2f} and \
{1:.2f} inches, respectively \n' .format(average_x_error, std_x_error))
print('The average error and standard deviation in the y-direction is {0:.2f} and \
{1:.2f} inches, respectively \n' .format(average_y_error, std_y_error))

Enter x coordinate of box: 
10
Enter y coordinate of box: 
4
The predicted displacement is (-3.22, -3.34) inches 

The average error and standard deviation in the x-direction is -10.96 and 9.33 inches, respectively 

The average error and standard deviation in the y-direction is -9.06 and 9.77 inches, respectively 



In [152]:
stdy_tests

[0.10825317547305482,
 1.4142135623730951,
 2.48746859276655,
 0.5,
 0.44633927678392815,
 0.409839907768875,
 1.7173289580042608,
 2.5395312067387557]